# Natural Language Processing

Start met laden van de packages en onze zojuist gemaakte dataset

In [2]:
import pandas as pd

In [5]:
data = pd.read_csv('data/fiets.csv', index_col =0)

In [6]:
data.head(2)

,Unnamed: 0.1,Unnamed: 0,identifier,type,title,date,content,subcategory,category,Year,DL score,Length,spatial
33663,84265,108,http://resolver.kb.nl/resolve?urn=MMKB23:00140...,artikel,BUITENLAND. POLITIEK OVERZICHT.,1880/05/26 00:00:00,"’s Gravenhage, 25 Mei.’t Is te Berlijn een lie...",fiets,fiets,1880,71.212121,792,Landelijk
33674,84276,121,http://resolver.kb.nl/resolve?urn=ddd:01026592...,artikel,"Amsterdam, 31 Mei. Een Laatste Hoop.",1882/06/01 00:00:00,"Ofschoon de meeste pruisische dioden n"" weder'...",fiets,fiets,1882,71.364047,1107,Landelijk


## Count word frequencies

Als eerste, meeste simpele stap van tekst analyse, kunnen we kijken naar de meest voorkomende woorden in de artikelen. Onderstaande functie is geschreven om dit te doen. Maar als eerste stap moeten we de artikelen van de kolom 'content' omzetten naar een lijst met woorden per artikel. 

In [69]:
data['content list'] = data.content.apply(lambda x: x.lower().split())

In [70]:
def word_counter(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

We kunnen de functie uitproberen:

In [71]:
values_count = word_counter(data['content list'])

In [72]:
values_count

de                  20041
van                 12865
het                 10233
en                   9962
een                  8878
                    ...  
overwicht               1
regeeren                1
ondergeschikte          1
veldtocht,              1
noordzee-kanaal.        1
Length: 62559, dtype: int64

Zoals je ziet zijn de eerste woorden allemaal woorden die inhoudelijk niet veel zeggen over de artikelen. Deze woorden, die veel voorkomen in onze taal maar inhoudelijk weinig zinnigs zeggen, noemen we stopwoorden. We kunnen een stopwoordenlijst gebruiken om deze woorden weg te filteren. 

In [100]:
with open('data/stopwords-nl.txt', 'r') as f:
    stopwords = f.read().split("\n")

In [101]:
## Don't alter
def word_counter_stopword(dataframe_column):
    full_list = []
    for elemnt in dataframe_column:
        full_list += elemnt
        
    full_list = [i for i in full_list if i not in stopwords]
    
    values_count = pd.Series(full_list).value_counts()
    return values_count

In [102]:
values_count = word_counter_stopword(data['content list'])

In [103]:
values_count.head(10)

—         1437
heer       883
dagen      528
plaats     436
groote     433
tijd       394
geheel     351
jaar       332
man        326
f          305
dtype: int64

# Spacy

Naast een naieve worcount, zijn er ook andere opties mogelijk. Voor het vervolg van deze workshop gaan we gebruik maken van de Spacy package, die gebruikt wordt voor Natural language processing. 


In [123]:
import spacy
from spacy import displacy
from collections import Counter

In [110]:
nlp = spacy.load("nl_core_news_sm")

In [111]:
data = data.dropna(subset=['content'])

In [112]:
def process_text(text):
    return nlp(text)

def flatten(xss):
    return [x for xs in xss for x in xs]

In [113]:
## deze code als comment omdat het lang duurt om te draaien
data["doc"] = data["content"].apply(process_text)

In [114]:
## Deze code ook als comment om te laten zien hoe je het model kan opslaan
import pickle

with open('data/fiets_nlp.pkl', 'wb') as f:
    pickle.dump(data, f)

In [119]:
with open('data/fiets_nlp.pkl', 'rb') as f:
    fiets_nlp = pickle.load(f)

In [120]:
fiets_nlp.head(2)

,Unnamed: 0.1,Unnamed: 0,identifier,type,title,date,content,subcategory,category,Year,DL score,Length,spatial,content list,doc
33663,84265,108,http://resolver.kb.nl/resolve?urn=MMKB23:00140...,artikel,BUITENLAND. POLITIEK OVERZICHT.,1880/05/26 00:00:00,"’s Gravenhage, 25 Mei.’t Is te Berlijn een lie...",fiets,fiets,1880,71.212121,792,Landelijk,"[’s, gravenhage,, 25, mei.’t, is, te, berlijn,...","(’s, Gravenhage, ,, 25, Mei.’t, Is, te, Berlij..."
33674,84276,121,http://resolver.kb.nl/resolve?urn=ddd:01026592...,artikel,"Amsterdam, 31 Mei. Een Laatste Hoop.",1882/06/01 00:00:00,"Ofschoon de meeste pruisische dioden n"" weder'...",fiets,fiets,1882,71.364047,1107,Landelijk,"[ofschoon, de, meeste, pruisische, dioden, n"",...","(Ofschoon, de, meeste, pruisische, dioden, n, ..."


# Named Entity Recognition

Spacy slaat informatie over de gevonden named entities op in het doc item. Deze informatie staat opgeslagen in de kolom 'doc' en kan daar ook uitgehaald worden.

Spacy kent de volgende Named entities:

* PERSON:      People, including fictional.
* NORP:        Nationalities or religious or political groups.
* FAC:         Buildings, airports, highways, bridges, etc.
* ORG:         Companies, agencies, institutions, etc.
* GPE:         Countries, cities, states.
* LOC:         Non-GPE locations, mountain ranges, bodies of water.
* PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
* EVENT:       Named hurricanes, battles, wars, sports events, etc.
* WORK_OF_ART: Titles of books, songs, etc.
* LAW:         Named documents made into laws.
* LANGUAGE:    Any named language.
* DATE:        Absolute or relative dates or periods.
* TIME:        Times smaller than a day.
* PERCENT:     Percentage, including ”%“.
* MONEY:       Monetary values, including unit.
* QUANTITY:    Measurements, as of weight or distance.
* ORDINAL:     “first”, “second”, etc.
* CARDINAL:    Numerals that do not fall under another type.

Je kan ook de spacy module vragen voor uitleg over een bepaalde Named Entity:

In [66]:
spacy.explain('GPE')

'Countries, cities, states'

Je kan per artikel de gevonden Named Entities tonen. Deze worden dan met verschillende kleuren weergegeven. 

In [126]:
doc = fiets_nlp['doc'].to_list()[2]

In [128]:
displacy.serve(doc, style = "ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [23/Jun/2025 22:16:07] "GET / HTTP/1.1" 200 37608


Shutting down server on port 5000.


In [131]:
## Zelf doen: Bekijk een random andere regel

Bekijk het op: http://localhost:5000/

Naast de named entities op deze manier tonen, kun je er ook verdere analyses mee doen. Je kan ze bijvoorbeeld per category opslaan als losse kolommen in je dataframe

Hieronder staat een functie waarmee je Named entities van een bepaalde soort uit het doc item kan halen. Deze worden vervolgens als een lijst opgeslagen in een nieuwe kolom in het dataframe. 

In [133]:
def get_gpe(doc, entity):
    return [ent.text for ent in doc.ents if ent.label_ == entity]

Vervolgens kun je deze functie gebruiken op je dataframe. 
Deze functie heeft als input de doc item van het dataframe, en de gekozen named entity.

In onderstaande voorbeeld wordt de categroy GPE eruit gehaald. 

In [134]:
fiets_nlp['GPE'] = fiets_nlp['doc'].apply(lambda x: get_gpe(x, 'GPE'))

Vervolgens kun je deze kolom weer gebruiken om de meest voorkomende GPE eruit te halen. 

In [135]:
values_count = word_counter(fiets_nlp['GPE'])

In [138]:
values_count.head(10)

Parijs       282
Amsterdam    186
Londen       163
Frankrijk    163
Engeland     161
Berlijn       89
Utrecht       83
Amerika       79
Nederland     78
Brussel       71
dtype: int64

In [ ]:
## Zelf doen
## Maak kolommen aan voor de entities ......

Nu je een aantal Named Entities hebt, kun je de datatset weer in twee splitsen (regionaal en landelijk) en kijken of detop 10 meest voorkomende woorden overeen komen.

In [140]:
regionaal = fiets_nlp[fiets_nlp['spatial'] == 'Regionaal/lokaal']

In [141]:
## Zelf doen
landelijk = fiets_nlp[fiets_nlp['spatial'] == 'Landelijk']

In [142]:
values_count = word_counter(regionaal['GPE'])
values_count.head(10)

Parijs       244
Amsterdam    161
Frankrijk    136
Londen       136
Engeland     134
Berlijn       74
Utrecht       73
Amerika       73
Nederland     65
Europa        61
dtype: int64

In [143]:
values_count = word_counter(landelijk['GPE'])
values_count.head(10)

Parijs       38
Londen       27
Engeland     27
Frankrijk    27
Amsterdam    25
Brussel      17
Berlijn      15
Nederland    13
België       12
Weenen       10
dtype: int64

In [144]:
## Zelf doen: andere kolommen word count vergelijken

# Part of speech tagging

Uitleg part of speech tagging en wat het inhoud

In [146]:
doc = fiets_nlp['doc'].to_list()[2]

In [147]:
displacy.serve(doc, style = "dep")

C:\Users\mirjam\anaconda3\lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [23/Jun/2025 22:33:26] "GET / HTTP/1.1" 200 1225026


Shutting down server on port 5000.


Bekijk het op: http://localhost:5000/

Ook voor part of speech tagging kun je kolommen maken in je dataframe met categorieën. 
Hiervoor is onderstaande functie geschreven:

In [153]:
def get_pos(doc, pos_tag):
    return [token for token in doc if token.pos_ == pos_tag]

Bij de Naed entities hebben we de top 10 meest voorkomende entities per categorie vergeleken tussen regionaal en lokaal. Voor de Part of Speech tagging gaan we kijken hoeveel procent van een artikel uit bepaalde woordsoorten bestaat. Hiervoor berekenen we eerst het aantal voorkomens per woordsoort, en daarna delen we dit door de lengte van het artikel. We gebruiken hiervoor het totale dataframe. 

Hieronder laten we deze methode zien in 3 stappen, voor zelfstandige naamwoorden (nouns)

Stap 1:

In [156]:
fiets_nlp['noun'] = fiets_nlp['doc'].apply(lambda x: get_pos(x, 'NOUN'))

Stap 2:

In [158]:
fiets_nlp['noun_count'] = fiets_nlp['noun'].apply(lambda x: len(x))

Stap 3:

In [161]:
fiets_nlp['noun_perc'] = (fiets_nlp['noun_count'] / fiets_nlp['Length']) * 100

In [163]:
fiets_nlp.head(2)

,Unnamed: 0.1,Unnamed: 0,identifier,type,title,date,content,subcategory,category,Year,DL score,Length,spatial,content list,doc,GPE,noun,noun_count,noun_perc
33663,84265,108,http://resolver.kb.nl/resolve?urn=MMKB23:00140...,artikel,BUITENLAND. POLITIEK OVERZICHT.,1880/05/26 00:00:00,"’s Gravenhage, 25 Mei.’t Is te Berlijn een lie...",fiets,fiets,1880,71.212121,792,Landelijk,"[’s, gravenhage,, 25, mei.’t, is, te, berlijn,...","(’s, Gravenhage, ,, 25, Mei.’t, Is, te, Berlij...","[Berlijn, Rome, Rome, nagaat, Granlle, Berlijn...","[Mei.’t, verwarring, wetsontwerp, overweging, ...",153,19.318182
33674,84276,121,http://resolver.kb.nl/resolve?urn=ddd:01026592...,artikel,"Amsterdam, 31 Mei. Een Laatste Hoop.",1882/06/01 00:00:00,"Ofschoon de meeste pruisische dioden n"" weder'...",fiets,fiets,1882,71.364047,1107,Landelijk,"[ofschoon, de, meeste, pruisische, dioden, n"",...","(Ofschoon, de, meeste, pruisische, dioden, n, ...","[Landdag, TaS, Pruisen, niag, Bismarck, Nergen...","[dioden, medewerking, bisschop, vrede, tussche...",218,19.692864


In [164]:
columns = ['noun_perc']

In [165]:
fiets_nlp.groupby(['spatial'])[columns].mean()

,noun_perc
spatial,
Landelijk,20.788330
Regionaal/lokaal,20.131937


In [166]:
fiets_nlp.groupby(['spatial'])[columns].median()

,noun_perc
spatial,
Landelijk,20.693037
Regionaal/lokaal,20.164609
